## Project PD2

### import packages

In [40]:
import datascience as ds
from datascience import *
import numpy as np
from collections import Counter
from graphviz import Source
import pandas as pd
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, classification_report
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.externals import joblib
%matplotlib inline

### tweets data loaded into Jupyter Notebook as Table object

In [41]:
df1 = ds.Table.read_table('Climate1SupportiveLevel.csv', sep=',')
df2 = ds.Table.read_table('ClimateBalancedDS2.csv', sep=',')
df = df1.append(df2)
# df = ds.Table.read_table('MergedLabelledData.csv', sep=',')
test_index = np.random.choice(df.num_rows, 100, replace=False)
train_val_index = [i for i in np.arange(df.num_rows) if i not in test_index]
test_data = df.take[test_index]
df = df.take[train_val_index]
X = list(df['Text'])
y = list(df['Support'])
test_X = list(test_data['Text'])
test_y = list(test_data['Support'])

### A description of all model enhancements incorporated into the construction of PD2.






#### used parameters

- CountVector
    - token_pattern = "(?!RT|rt|\d+)[@#]*[\w\'_-]{2,100}"
    - analyzer = 'word'
    - stop_words = the top 30 most common words in DS1+DS2 tweets 
    - min_df = 10
    - ngram_range=(1,2)
- DecisionTree
    - criterion='entropy'
    - max_depth = 10
    - min_samples_leaf = 2

### A description of all model parameters you tried and the associated Stratified k-fold cross validation results for each model parameter choice based on the combined data set (DS1 + DS2)

 - max_depth=range(1, 15)
 - stop_words:
     - default in CountVectorizer: 'english'
     - user_defined: ['a', 'an', 'the', 'it', 'is', 'are', 'be', 'of', 'this', 'that', 'RT', 'rt','https']
     - the top 30 most common words in DS1+DS2 tweets (chosen)
     
code and test results as follows:

### Check whether the data distribution is balanced

In [42]:
def check(sentiment, index, note='training'):
    if sentiment==0:
        label = 'not supportive'
    else:
        label = 'supportive'
    print('There are {} '.format(df.take(index).where('Support', 
          are.equal_to(sentiment)).size[0][0])+label+' tweets in the '+note+' set.')

#### Model Building

In [43]:
def custom_split(train_index, test_index):
    trainingset = df.take(train_index)
    testingset = df.take(test_index)    
        
    X_train= list(trainingset['Text'])
    y_train= list(trainingset['Support'])
    X_test= list(testingset['Text'])
    y_test= list(testingset['Support'])
    
    return X_train, X_test, y_train, y_test

### classifier

In [44]:
def classifier(X_train, y_train, X_test, fold, max_depth, min_samples_leaf, stop_words, overfit_risk=False):
    # token_pattern='(([#@]|[0-9]|[a-z]|[A-Z])+)'
    clf = Pipeline(
        [
            ('vect', CountVectorizer(token_pattern="(?!RT|rt|\d+)[@#]*[\w\'_-]{2,100}",
                                     analyzer = 'word',
                                     stop_words = stop_words,
                                     min_df = 10,
                                     ngram_range=(1,2))),
            ('clf', DecisionTreeClassifier(criterion='entropy',
                                           random_state = 100,
                                           max_depth = max_depth,
                                           min_samples_leaf = min_samples_leaf))
        ])
    clf.fit(X_train, y_train)
    feature_names = clf.named_steps['vect'].get_feature_names()
    try:
        dot_data = tree.export_graphviz(clf.named_steps['clf'], out_file=None, 
                                        feature_names=feature_names)
        graph = Source(dot_data)
        graph.render('ClimateClassifier-Fold_{}'.format(fold))
    except Exception as e:
        print(e)
    predicted_y_train = clf.predict(X_train)
    predicted_y_test = clf.predict(X_test)
    # save as pickle
    if overfit_risk:
        joblib.dump(clf, 'ClimateTeam7PD2_maxdepth{}.pkl'.format(max_depth))
    else:
        joblib.dump(clf, 'ClimateTeam7PD2.pkl')
    return predicted_y_train, predicted_y_test

### evaluation

In [45]:
def eval_results(predicted_y_train, y_train, predicted_y_test, y_test, fold):
    accuracy_s = accuracy_score(y_test, predicted_y_test)
    precision_s = precision_score(y_test, predicted_y_test)
    recall_s = recall_score(y_test, predicted_y_test)
    f1_s = f1_score(y_test, predicted_y_test)
    cm_train = confusion_matrix(y_train, predicted_y_train)
    cm_test = confusion_matrix(y_test, predicted_y_test)  
    
    print('Accuracy Score:', accuracy_s)
    print("Precision Score:", precision_s)
    print("Recall Score:", recall_s)
    print("f1 Score:", f1_s)
    print('confusion_matrix of training set is: \n', cm_train, '\n')
    print('confusion_matrix of testing set is: \n', cm_test, '\n')
    print(classification_report(y_test, predicted_y_test))
    
    classes = ['not supportive', 'supportive']
    plt.subplot(2, 5, fold)
    sns.heatmap(cm_train, annot=True, cmap='Blues', yticklabels=classes, xticklabels=classes)
    plt.title('Fold {}: confusion matrix of training set'.format(fold))
    plt.subplot(2, 5, fold+5)
    sns.heatmap(cm_test, annot=True, cmap='Blues', yticklabels=classes, xticklabels=classes)
    plt.title('Fold {}: confusion matrix of testing set'.format(fold))
    return accuracy_s, precision_s, recall_s, f1_s

### k-fold

In [46]:
def k_fold_evaluate(X, y, max_depth, min_samples_leaf, stop_words, print_eval=True, overfit_risk=False):
    # initialization
    accuracy = []
    precision = []
    recall=[]
    f1 = []
    fold = 1
    skf = StratifiedKFold(n_splits=5, random_state=1, shuffle= True)
    
    # build model and collect results
    for val_index, test_index in skf.split(X, y):
        X_train, X_val, y_train, y_val = custom_split(val_index, test_index)
        
        predicted_y_train, predicted_y_val = classifier(X_train=X_train, y_train=y_train, 
                                                          X_test=X_val, fold=fold,
                                                          max_depth = max_depth,
                                                          min_samples_leaf = min_samples_leaf,
                                                          stop_words = stop_words,
                                                          overfit_risk=overfit_risk)
        metrics_df={}
        if print_eval:
            print('\nFold: {}'.format(fold))
            accuracy_s, precision_s, recall_s, f1_s = eval_results(predicted_y_train, y_train, predicted_y_val, y_val, fold)

            accuracy.append(accuracy_s)
            precision.append(precision_s)
            recall.append(recall_s)
            f1.append(f1_s)

            metrics_df = pd.DataFrame(
                        {
                            'accuracy': accuracy,
                            'precision': precision,
                            'recall':recall,
                            'f1':f1
                        }
                    )
            fold += 1
    return metrics_df

### Tests: max_depth : 1-15

In [47]:
df1 = ds.Table.read_table('Climate1SupportiveLevel.csv', sep=',')
df2 = ds.Table.read_table('ClimateBalancedDS2.csv', sep=',')
df = df1.append(df2)
test_index = np.random.choice(df.num_rows, 100, replace=False)
train_val_index = [i for i in np.arange(df.num_rows) if i not in test_index]
test_data = df.take[test_index]
df = df.take[train_val_index]
X = list(df['Text'])
y = list(df['Support'])
test_X = list(test_data['Text'])
test_y = list(test_data['Support'])

#### test1: stop-words: default

In [48]:
f1_lst_test1 = []
f1_lst_train1 = []
for d in range(1, 16):
    k_fold_evaluate(X, y, max_depth=d, min_samples_leaf=2, stop_words='english', print_eval=False, overfit_risk=True)
    clf_tmp = joblib.load('ClimateTeam7PD2_maxdepth{}.pkl'.format(d))
    print('maxdepth=', d)
    # test
    y_pred = clf_tmp.predict(test_X)
    print('test f1')
    print(f1_score(y_pred=y_pred, y_true=test_y))
    f1_lst_test1.append(f1_score(y_pred=y_pred, y_true=test_y))
    # train_val
    y_pred = clf_tmp.predict(X)
    print('train f1')
    print(f1_score(y_pred=y_pred, y_true=y))
    f1_lst_train1.append(f1_score(y_pred=y_pred, y_true=y))

maxdepth= 1
test f1
0.5454545454545454
train f1
0.6031746031746031
maxdepth= 2
test f1
0.6607142857142857
train f1
0.6836248012718602
maxdepth= 3
test f1
0.6782608695652174
train f1
0.6907020872865275
maxdepth= 4
test f1
0.6724137931034483
train f1
0.6986899563318778
maxdepth= 5
test f1
0.6434782608695652
train f1
0.7041198501872659
maxdepth= 6
test f1
0.6434782608695652
train f1
0.7078757452149357
maxdepth= 7
test f1
0.6434782608695652
train f1
0.7139303482587065
maxdepth= 8
test f1
0.6551724137931033
train f1
0.7217257318952235
maxdepth= 9
test f1
0.6551724137931033
train f1
0.7243076923076922
maxdepth= 10
test f1
0.6666666666666665
train f1
0.7271039603960396
maxdepth= 11
test f1
0.6666666666666665
train f1
0.7297633872976339
maxdepth= 12
test f1
0.6434782608695652
train f1
0.7315855181023719
maxdepth= 13
test f1
0.6551724137931033
train f1
0.732766761095373


KeyboardInterrupt: 

In [ ]:
plt.plot(np.arange(1,16), f1_lst_test1, label='f1 of testset')
plt.plot(np.arange(1,16), f1_lst_train1, label='f1 of trainset')
plt.xlabel('max_depth')
plt.title('Double check the overfitting risk')
plt.legend(loc='lower right')
plt.show()

#### test2: stop_words: ['a', 'an', 'the', 'it', 'is', 'are', 'be', 'of', 'this', 'that', 'RT', 'rt','https']

In [ ]:
f1_lst_test2 = []
f1_lst_train2 = []
for d in range(1, 16):
    k_fold_evaluate(X, y, max_depth=d, min_samples_leaf=2, 
                    stop_words=['a', 'an', 'the', 'it', 'is', 'are', 'be', 'of', 'this', 'that', 'RT', 'rt','https'], 
                    print_eval=False, overfit_risk=True)
    clf_tmp = joblib.load('ClimateTeam7PD2_maxdepth{}.pkl'.format(d))
    print('maxdepth=', d)
    # test
    y_pred = clf_tmp.predict(test_X)
    print('test f1')
    print(f1_score(y_pred=y_pred, y_true=test_y))
    f1_lst_test2.append(f1_score(y_pred=y_pred, y_true=test_y))
    # train_val
    y_pred = clf_tmp.predict(X)
    print('train f1')
    print(f1_score(y_pred=y_pred, y_true=y))
    f1_lst_train2.append(f1_score(y_pred=y_pred, y_true=y))

In [ ]:
plt.plot(np.arange(1,16), f1_lst_test2, label='f1 of testset')
plt.plot(np.arange(1,16), f1_lst_train2, label='f1 of trainset')
plt.xlabel('max_depth')
plt.title('Double check the overfitting risk')
plt.legend(loc='lower right')
plt.show()

#### test3: stop_words: most common words in DS1+DS2 tweets

In [ ]:
stop_w = [i[0] for i in Counter([word for sentence in X for word in sentence.split() 
                                 if 'nike' not in word.lower() and 'colin' not in word.lower() and 'kaepernick' not in word.lower()
                                 and word.isalpha() 
                                 and len(word)>1]).most_common()[:30]]
stop_w

In [ ]:
f1_lst_test3 = []
f1_lst_train3 = []
for d in range(1, 16):
    k_fold_evaluate(X, y, max_depth=d, min_samples_leaf=2, 
                    stop_words=stop_w, 
                    print_eval=False, overfit_risk=True)
    clf_tmp = joblib.load('ClimateTeam7PD2_maxdepth{}.pkl'.format(d))
    print('maxdepth=', d)
    # test
    y_pred = clf_tmp.predict(test_X)
    print('test f1')
    print(f1_score(y_pred=y_pred, y_true=test_y))
    f1_lst_test3.append(f1_score(y_pred=y_pred, y_true=test_y))
    # train_val
    y_pred = clf_tmp.predict(X)
    print('train f1')
    print(f1_score(y_pred=y_pred, y_true=y))
    f1_lst_train3.append(f1_score(y_pred=y_pred, y_true=y))

In [ ]:
plt.plot(np.arange(1,16), f1_lst_test3, label='f1 of testset')
plt.plot(np.arange(1,16), f1_lst_train3, label='f1 of trainset')
plt.xlabel('max_depth')
plt.title('Double check the overfitting risk')
plt.legend(loc='lower right')
plt.show()

## Final result

In [ ]:
plt.figure(figsize=(25, 8))
metrics_df = k_fold_evaluate(X, y, max_depth=15, min_samples_leaf=2, 
                    stop_words=stop_w, 
                    print_eval=True, overfit_risk=False)
plt.show()

In [ ]:
metrics_df

In [ ]:
clf2 = joblib.load('ClimateTeam7PD2.pkl')

In [ ]:
df1 = ds.Table.read_table('Climate1SupportiveLevel.csv', sep=',')
df2 = ds.Table.read_table('ClimateBalancedDS2.csv', sep=',')
df = df1.append(df2)
X = list(df['Text'])
y = list(df['Support'])
y_pred = clf_tmp.predict(X)
f1_score(y, y_pred)